# Attack on TCP

### Sample TCP program (client and server) 
```c
#include <stdio.h>
#include <string.h>
#include <unistd.h> 
#include <sys/socket.h>
#include <netinet/ip.h>

int main() 
{
    // step 1: create a socket
    int sockfd = socket(AF_INET, SOCK_STREAM, 0); 
    
    // step 2: set the destination information 
    struct sockaddr_in dest;
    memset(&dest, 0, sizeof(struct sockaddr_in));
    dest.sin_family = AF_INET;
    dest.sin_addr.s_addr = inet_addr("10.11.0.45");
    dest.sin_port = htons(9090); 
    
    // step3: connect to the server
    connect(sockfd, (struct sockaddr *) &dest, sizeof(struct sockaddr_in)); 
    
    // step4: send data to the server
    char *buffer1 = "hello server!\n");
    char *buffer2 = "hello again!\n");
    write(sockfd, buffer1, strlen(buffer1)); 
    
    write(sockfd, buffer2, strlen(buffer2)); 
    
    // step5: close the connection 
    close(sockfd);
    return 0; 
}
```



Unlike UDP, TCP sets up end-to-end connections, and therefore, a TCP client has following steps:
* __Step1: Create a socket__: in particular socket is set to be SOCK_STREAM
* __Step2: Set the destination information__; this is the same as UDP 
* __Step3: Connect to the sever__: this shows the connection-oriented property of TCP. A connection is defined by the 4-tuple (source IP, source port, destination IP, destination port)
* __Step4: send and receive data__; both ends can send and receive data, via methods such as `write(), send(), sendto(), sendmsg(), read(), recv(), recvfrom(), recvmsg()` system calls 
* __Step5: Close connection__: using the `close()` system call.


```c 
// TCP server
#include <stdio.h>
#include <string.h>
#include <sys/socket.h>
#include <netinet/ip.h>
#include <unistd.h>

int main() 
{
    int sockfd, newsockfd;
    struct sockaddr_in my_addr, client_addr; 
    char buffer[100];
    
    // Step1: create a socket
    sockfd = socket(AF_INET, SOCK_STREAM, 0); 
    
    // step2: bind to a port number 
    memset(&my_addr, 0, sizeof(struct sockaddr_in)); 
    my_addr.sin_family = AF_INET;
    my_addr.sin_port = htons(9090);
    
    bind(sockfd, (struct sockaddr *) &my_addr, sizeof(struct sockaddr_in_)); 
    
    // step3: listen for connections 
    listen(sockfd, 5);
    
    // step4: accept a connection request 
    int client_len = sizeof(client_addr); 
    newsockfd = accept(sockfd, (struct sockaddr *) &client_addr, &client_len); 
    
    // step5: read data from connection
    memset(buffer, 0, sizeof(buffer));
    int len = read(newsockfd, buffer, 100); 
    printf("received %b bytes: %s\n", len, buffer); 
    
    // step6: close the connection
    close(newsockfd);
    close(sockfd); 
    
    return 0; 
}
```

The phases for a TCP server are: 
* __Step1: create a socket__
* __Step2: Bind to a port number__. Register a port number to receive packets, so that OS knows which application to turn to. `bind()` is used for binding a port with a  sockaddr struct. 
* __Step3: Listen for Connections__. Once socket is setup, TCP programs call `listen()` system call to wait for connections. The call __is non-block__, so it tells system that the application is ready for receiving connection requests. (it is event based). Once a connection request is received, the OS will go through TCP three-way handshake protocol with the client to establish a connection. second argument specifies maximum size of queued connection requests. If queue is full, no further connection can be made. 
* __Step4: Accept a Connection request__: `accept()` is called to extract the first connection request in the queue, create a new socket, and return a new file descriptor referring to that socket. The call __will block__ the calling application if there are no pending connections, unless the socket is marked as non-blocking.
* __Step5: Send and receive data__: This is similar to the client program. 

### Sample multiple connections TCP program
```c
// only after the listen syscall needs change 
listen(sockfd, 5);

int client_len = sizeof(client_addr);
// use multiprocessing for multiple connections 
while (1) {
    newsockfd = accept(sockfd, (struct sockaddr *) &client_addr, &client_len);
    if (fork() == 0) {
        close(sockfd);
        
        memset(buffer, 0, sizeof(buffer));
        int len = read(newsockfd, buffer, 100);
        printf("Received %d bytes. \n %s \n", len, buffer);
        
        close(newsockfd);
        return 0;
    }
    else {
        close(newsockfd);
    }
}
```

### TCP Header Examination 
![TCP](image_files/tcp.png)

The TCP part of the IP packet is called _TCP segment_. It starts with TCP header, followed by a payload. <br>
* __Source and Destination port__: 16 bits, specify source and dest port number 
* __Sequence Number__: 32 bits, specifies the sequence number of the first octet in this TCP segment. If `SYN` bit is set, then sequence number is the initial sequence number.
* __Acknowledge Number__: 32 bits, only valid when `ACK` is set. It contains the value of the next sequence number expected by the sender of this segment. 
* __Header Length__: The length of the TCP header is measured by the number of 32-bit words in the header, so we need to multiply the value in this field by 4 to get the number of octets in the TCP header. 
* __Reserved__: The field not used.
* __Code bits__: 6 bits, `SYN, FIN, ACK, RST, PSH, URG`, all for different functionalities. 
* __Window__: 16 bits, used for flow control and congestion control 
* __Checksum__: computed from TCP pseudoheader 
* __Urgent Pointer__: 16 bits, used when priority and emergency is desired. The pointer specifies where the urgent data ends and when the TCP data starts.
* __Options__: TCP segments can carry a variable length of options, which provide a way to deal with the limitations of the original header. 

### SYN Flood Attack
#### Three way handshake 
![3-way](image_files/three.png)

* First, the client sends a special packet called `SYN` packet to the server, using a randomly generated number as its sequence number. The packet is called `SYN` packet because `SYN` bit in TCP header is set to one. 
* Second, after the server receives the packet, it replies with a `SYN+ACK` packet, with both `SYN` and `ACK` bits set to one. The server also chooses its own randomly generated number as its initial sequence number.
* Third, when the client get this packet, it sends out a `ACK` packet to conclude the handshake.

One caveat is that when the server receives the initial `SYN` packet, it uses a special data structure called Transmission Control Block (__TCB__) to store the information about this connection. <br>
At this step, the connection is not fully established yet; it is called a __half-open conneciton__. Now, only the client-to-server direction of the connection is confirmed, and the server-to-client direction has not been initialized yet. After the server gets the `ACK` packet from the client, it will take this TCB out of the queue, and store it to a different place. <br>
If the final `ACK` packe does not come, the server will resend its `SYN + ACK` packet. If the final ACK packet never comes, the TCB stored in the half open connection queue will eventually time out and be discarded.